In [426]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers
import keras

In [427]:
def training(len_input, embed_space_size, embed_action_size):
    '''define layer
    embed_space_size: space種類數
    embed_action_size: action種類數
    len_input: (space預計輸入維度, action預計輸入維度, others預計輸入維度)
    '''
    input_space = keras.Input(shape=len_input[0], name='Space_input')
    input_action = keras.Input(shape=len_input[1], name='Action_input')
    input_other = keras.Input(shape=len_input[2], name='Other_input')

    space_embedding = layers.Embedding(input_dim=embed_space_size, output_dim=4, mask_zero=True, name='Space_embedding')
    action_embeding = layers.Embedding(input_dim=embed_action_size, output_dim=4, mask_zero=True, name='Action_embedding')
    
    layer_reshape_space = layers.Reshape((4,), input_shape=(len_input[0], 4), name='reshape_space')
    layer_reshape_action = layers.Reshape((4,), input_shape=(len_input[1], 4), name='reshape_action')

    input_concat_embedding = layers.Concatenate(name='Input_merging', axis=1)
    layer_dense = layers.Dense(units=3, activation='softmax')

    '''define forward'''
    inputs = [input_space, input_action, input_other]
    embeded_space = space_embedding(input_space)
    embeded_action = action_embeding(input_action)

    embeded_space = layer_reshape_space(embeded_space)
    embeded_action = layer_reshape_action(embeded_action)

    embeded_input = input_concat_embedding([embeded_space, embeded_action, input_other])
    output_prob = layer_dense(embeded_input)

    model = keras.Model(inputs=inputs, outputs=output_prob, name='classifier')

    return model

---

prepare data

In [428]:
def split_data(space_tensor, action_tensor, others_tensor, label):
    l = len(label)
    split_persentage = int(l*0.7)

    train_space = space_tensor[:split_persentage]
    train_action = action_tensor[:split_persentage]
    train_others = others_tensor[:split_persentage]
    train_label = label[:split_persentage]

    test_space = space_tensor[split_persentage:]
    test_action = action_tensor[split_persentage:]
    test_others = others_tensor[split_persentage:]
    test_label = label[split_persentage:]

    train_x = [train_space, train_action, train_others]
    train_y = train_label

    test_x = [test_space, test_action, test_others]
    test_y = test_label

    return train_x, train_y, test_x, test_y

In [429]:
def prepare_data(df):
    input = df[['Team', 'No', 'Space', 'Action']].copy()
    label = df[['Erros', 'Score']].copy().astype('float')

    space = input[['Space']].astype('float')
    action = input[['Action']].astype('float')
    others = input[['Team', 'No']].copy()

    others = others.astype({'No': 'str'})
    others_dummy = pd.get_dummies(others).astype('float')

    label.loc[:, 'Nothing'] = 0.0
    label.loc[(label['Erros'] == 0) & (label['Score'] == 0), 'Nothing'] = 1.0

    space_tensor = tf.convert_to_tensor(space)
    action_tensor = tf.convert_to_tensor(action)

    others_tensor = tf.convert_to_tensor(others_dummy)
    label_tensor = tf.convert_to_tensor(label)
    
    train_x, train_y, test_x, test_y = split_data(space_tensor, action_tensor, others_tensor, label_tensor)

    return train_x, train_y, test_x, test_y

In [430]:
df = pd.read_csv('df.csv')

In [431]:
df.groupby(by=['Score']).count()

,Team,No,Space,Action,Erros,Attempts
Score,,,,,,
0.0,4825,4825,4825,4825,4825,4825
1.0,697,697,697,697,697,697


In [432]:
train_x, train_y, test_x, test_y = prepare_data(df)

print(len(train_y), len(test_y))

3865 1657


In [433]:
embed_space_size = 15
embed_action_size = 9
model = training(
    (1, 1, train_x[2].shape[1]), 
    embed_space_size, 
    embed_action_size
    )
    
model.summary()

Model: "classifier"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Space_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
Action_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
Space_embedding (Embedding)     (None, 1, 4)         60          Space_input[0][0]                
__________________________________________________________________________________________________
Action_embedding (Embedding)    (None, 1, 4)         36          Action_input[0][0]               
_________________________________________________________________________________________

In [434]:
# regularizer = tf.keras.regularizers.l2(0.01)
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['AUC']
epochs = 20
callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [437]:
pred = model.fit(train_x, train_y, epochs=epochs, callbacks=[callbacks])

Epoch 1/20
121/121 [==============================] - 0s 1ms/step - loss: 0.5879 - auc: 0.9002
Epoch 2/20
121/121 [==============================] - 0s 974us/step - loss: 0.5876 - auc: 0.9003
Epoch 3/20
121/121 [==============================] - 0s 1ms/step - loss: 0.5876 - auc: 0.9003
Epoch 4/20
121/121 [==============================] - 0s 1ms/step - loss: 0.5871 - auc: 0.9005
Epoch 5/20
121/121 [==============================] - 0s 1ms/step - loss: 0.5870 - auc: 0.9004
Epoch 6/20
121/121 [==============================] - 0s 1ms/step - loss: 0.5868 - auc: 0.9005
Epoch 7/20
121/121 [==============================] - 0s 1ms/step - loss: 0.5866 - auc: 0.9006
Epoch 8/20
121/121 [==============================] - 0s 1ms/step - loss: 0.5865 - auc: 0.9006
Epoch 9/20
121/121 [==============================] - 0s 1ms/step - loss: 0.5865 - auc: 0.9006
Epoch 10/20
121/121 [==============================] - 0s 969us/step - loss: 0.5863 - auc: 0.9007
Epoch 11/20
121/121 [========================

In [438]:
result = model.evaluate(test_x, test_y)
print(model.metrics_names)
print(result)

52/52 [==============================] - 0s 1ms/step - loss: 0.5845 - auc: 0.8967
['loss', 'auc']
[0.5845266580581665, 0.8966692090034485]
